# Spliting the images/images path into train,test and validation , as the dataset in not provided in that format

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Define data path
data_dir = '/kaggle/input/bengali-food-58/ALLFOOD'

# Get list of image files and labels
image_files = []
labels = []
for class_label in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_label)
    for image_file in os.listdir(class_dir):
        image_files.append(os.path.join(class_dir, image_file))
        labels.append(class_label)

# Split data into training and testing sets
train_files, test_files, train_labels, test_labels = train_test_split(image_files, labels, test_size=0.1)

# Split training set into training and validation sets
train_files, valid_files, train_labels, valid_labels = train_test_split(train_files, train_labels, test_size=0.1)

# Create dataframes for the splits
train_df = pd.DataFrame({'filename': train_files, 'class': train_labels})
valid_df = pd.DataFrame({'filename': valid_files, 'class': valid_labels})
test_df = pd.DataFrame({'filename': test_files, 'class': test_labels})



# Importing necessary libraries

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.regularizers import l2
from tensorflow import keras
import numpy as np
from sklearn.metrics import classification_report

# Definind different parameters and data augmentation . Loading the datasets  using image-data generator 

In [ ]:

img_height = 224
img_width = 224
n_classes = 58

batch_size = 64

# train_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     rotation_range=20
   
    
# )
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=0.4,
    rotation_range=30,
    vertical_flip=True,
    horizontal_flip=True,
    width_shift_range=0.4,
    height_shift_range=0.3,
    brightness_range=[0.8, 1.2],
    channel_shift_range=20,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

# Generate training set from dataframe
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

# Generate validation set from dataframe
validation_generator = validation_datagen.flow_from_dataframe(
    valid_df,
    x_col='filename',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Generate testing set from dataframe
test_generator = validation_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)




# Importing the pre-trained res-net model from keras and freezing first 500 layers . We have decided to re-train the last few layers according to our dataset as the contain the most complex features .

In [ ]:
# inception V3 311 layers
densenet = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False,input_shape=(img_width,img_width,3))
for layer in densenet.layers[:500]:
    layer.trainable=False

In [ ]:
c=0
for layer in densenet.layers:
    c+=1
c

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Get the class labels from the generator
class_labels = train_generator.classes

# Compute class weights
class_weights = compute_class_weight('balanced',classes= np.unique(class_labels),y= class_labels)

# Convert class weights to a dictionary
# class_indices = train_generator.class_indices
# class_weights_dict = dict(zip(class_indices.values(), class_weights))
class_weights_dict = dict(zip(np.unique(class_labels), class_weights))

# Model Training 

In [ ]:
x = densenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(150,activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(n_classes,kernel_regularizer=regularizers.l2(0.01), activation='softmax')(x)
model = Model(inputs=densenet.input, outputs=output)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='./densenet_best_loss.hdf5', verbose=1, save_best_only=True)
from keras.callbacks import LearningRateScheduler
def learning_rate_scheduler(epoch, lr):
    initial_lr=0.0001
    warmup_epochs = 10 # Number of epochs for warm-up
    if epoch < warmup_epochs:
        # Gradually increase the learning rate
        warmup_factor = epoch / warmup_epochs
        lr = warmup_factor*initial_lr
        return lr
    else:
        return initial_lr


lr_scheduler = LearningRateScheduler(learning_rate_scheduler)
history=model.fit(train_generator,
        steps_per_epoch = len(train_generator) ,
        validation_data = validation_generator,
        validation_steps = len(validation_generator),
        epochs=60,
        verbose=1,
       callbacks=[ lr_scheduler,checkpointer],
         class_weight=class_weights_dict)
         
model.save('./densenet_best_Final.hdf5')

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
import matplotlib.pyplot as plt

epochs = range(1, len(train_loss) + 1)


import pandas as pd

loss_df = pd.DataFrame({'Epoch': epochs, 'Train Loss': train_loss, 'Validation Loss': val_loss})
loss_df.to_csv('inception_loss_data.csv', index=False)





plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Test Result

In [ ]:
# y_pred = model.predict(test_generator)
# y_pred = tf.argmax(y_pred, axis=1)
# print('Classification Report:')
# print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))

In [ ]:
y_pred = model.predict(test_generator)
y_pred = tf.argmax(y_pred, axis=1)
class_labels = list(test_generator.class_indices.keys())
print('Classification Report:')
print(classification_report(test_generator.classes, y_pred, labels=range(len(class_labels)), target_names=class_labels))

In [ ]:
import json

def switch_key_value_positions(class_indices):
    switched_indices = {v: k for k, v in class_indices.items()}
    return switched_indices

# Example usage
class_indices = test_generator.class_indices  # Assuming `test_generator` is your generator object

switched_indices = switch_key_value_positions(class_indices)

# Convert the switched indices to JSON format
json_data = json.dumps(switched_indices)

# Save the JSON data to a file
with open('/kaggle/working/densenet_indices.json', 'w') as file:
    file.write(json_data)

In [ ]:
from keras.models import load_model

# Load the model from HDF5 file
model = load_model('/kaggle/working/densenet_best_loss.hdf5')
y_pred = model.predict(test_generator)
y_pred = tf.argmax(y_pred, axis=1)
class_labels = list(test_generator.class_indices.keys())
print('Classification Report:')
print(classification_report(test_generator.classes, y_pred, labels=range(len(class_labels)), target_names=class_labels))